In [73]:
from __future__ import print_function

import pandas as pd
import numpy as np
import requests
from threading import Thread
import glob

In [74]:
files = glob.glob('./surveys/*.csv')
files

['./surveys/NASA-TLX Rating Scale (4,every mission debrief).csv',
 './surveys/NASA-TLX Weighted Scale (21).csv',
 './surveys/2nd Trust Survey (5,non-adaptive,every mission debrief).csv',
 './surveys/1st Trust Survey - (3).csv',
 './surveys/End of Experiment Survey (20).csv',
 './surveys/2nd Trust Survey (6,adaptive, every mission debrief).csv',
 './surveys/Demographic Survey (1).csv',
 './surveys/IPIP Survey (2).csv']

In [131]:
class upload_surveys:
    #requires a survey folder
    #requires a debrief folder
    def __init__(self,path="./surveys/*.csv"):
        self.path = path
        self.survey_column_name = "Survey Name"
        self.question_column_name = "Question"
        self.type_column_name = "Type"
        self.offered_answers_name = "Official Answers"
        iURL = 'http://192.168.1.11:8000/sql-temp/insert'
        threadpool = []
        for fname in glob.glob(self.path):
            t = Thread(target= self.upload_survey, args=(fname,iURL,))
            threadpool.append(t)
        for t in threadpool:
            t.start()
        for t in threadpool:
            t.join()
        
    def send_query(self,query,URL):
        iData = {
                    'sql': query,
                    'database': 'dbsurveys'
                }
        
        #print (iData)
        req = requests.post(url=URL, data=iData)
        #if "200" in req:
         #   print(query)
        #print(req)
        #print(req.text)
    def remove_survey (self,survey_id):
        iData = {
                    'survey_id': survey_id
                }
        URL = 'http://192.168.1.11:8000/survey/remove-survey'
        #print (iData)
        req = requests.post(url=URL, data=iData)

    def upload_survey(self,path,iURL):
        
        data = pd.read_csv(path)
        #print (data.iloc[0][0])
        #print (data.iloc[2][0])
        survey_name = data.iloc[0][0]
        survey_id = int(data.iloc[2][0])
        self.remove_survey(survey_id)
        print(list(data)[1])
        
        
        #Mr. Clean
        data.drop([list(data)[0]],axis=1,inplace=True)
        #data.replace(r'\s', '', regex=True,inplace=True)
        data[self.question_column_name] = data[self.question_column_name].str.strip(' ')
        data.replace('', np.nan, regex=True,inplace=True)
        data.dropna(axis=0,how='all',inplace=True)
        
        
        #will craete a new entry if it does not already exists
        survey_query =("INSERT INTO dbsurveys.survey (id, description)" + 
                        "SELECT * FROM (SELECT '{0}', '{1}') AS tmp " + 
                        "WHERE NOT EXISTS (" + 
                        "SELECT description FROM dbsurveys.survey " +
                        "WHERE description = '{1}') LIMIT 1;")
        survey_query = survey_query.format(survey_id,survey_name)

        question_query = "INSERT INTO dbsurveys.question (id, type, question_text) VALUE "
        offered_answer_query = "INSERT INTO dbsurveys.offered_answer (id, offered_answer_text) VALUE"
        survey_question_query = "INSERT INTO dbsurveys.survey_question (survey_id, question_id) VALUE "
        question_offered_answer_query =  "INSERT INTO dbsurveys.question_answer (question_id, offered_answer_id) VALUE "
        
        answers = []
        #print (data)
        for i, row in data.iterrows():
            index = i + survey_id*100
            
            
            question_text = str(row[self.question_column_name]).replace("'", "\\'")
            question_type = str(row[self.type_column_name]).replace("'", "\\'")
            
            offered_answer = ""
            if(pd.isnull(row[self.offered_answers_name])):
                offered_answer_text = ""
                offered_answer_query += "('{0}', '{1}')".format(index, offered_answer_text)
            else:
                offered_answer_text = str(row[self.offered_answers_name]).replace("'", "\\'")
                offered_answer_query += "('{0}','{1}')".format(index, offered_answer_text)
            question_query += "('{0}', '{1}', '{2}')".format(index,question_type, question_text)
            survey_question_query += "('{0}','{1}')".format(survey_id,index)
            question_offered_answer_query += "('{0}','{1}')".format(index,index)
            #if index > 5:
             #       break

            if i < len(data.index) - 1:
                question_query += ","
                offered_answer_query += ","
                survey_question_query += ","
                question_offered_answer_query += ","
          
           
        question_query += ";\n\n"
        offered_answer_query += ";\n\n"
        survey_question_query += ";\n\n"
        question_offered_answer_query += ";\n\n"

        #breaks post request for me
        query = survey_query+ question_query + offered_answer_query + survey_question_query + question_offered_answer_query    
        print ("path = "+ path + "\n\n" + question_query)
        self.send_query(survey_query , iURL)
        self.send_query(question_query , iURL)
        self.send_query(offered_answer_query , iURL)
        self.send_query(survey_question_query , iURL)
        self.send_query(question_offered_answer_query , iURL)
        
        
        #req = requests.post(url=iURL, data=iData)

In [132]:
path = "./surveys/*.csv"
upload_surveys()



Question
Question
Question
QuestionQuestion
Question

Question
path = ./surveys/NASA-TLX Weighted Scale (21).csv

INSERT INTO dbsurveys.question (id, type, question_text) VALUE ('2100', 'Intro', 'You will be presented with a series of pairs of rating scale titles (for example,  Effort vs. Mental Demand) and asked to choose which of the items was more important to your experience of workload in the tasks) that you just performed. 

Each pair of scale titles will appear one set at a time. 
Please select the Scale Title that represents the more important contributor to workload for the specific tasks you performed in this experiment.

You may click “begin survey” whenever you are ready.
'),('2101', 'Multiple', 'Please select the Scale Title that represents the more important contributor to workload.'),('2102', 'Multiple', 'Please select the Scale Title that represents the more important contributor to workload.'),('2103', 'Multiple', 'Please select the Scale Title that represents the more

<__main__.upload_surveys instance at 0x7f46fcbc50e0>

In [17]:
survey_id = 4
iData = {
            'survey_id': survey_id
        }
URL = 'http://ec2-52-24-126-225.us-west-2.compute.amazonaws.com:81/survey/remove-survey'
#print (iData)
req = requests.post(url=URL, data=iData)

In [62]:
df = pd.DataFrame({'a':['',' ','asasd']})
df

,a
0,
1,
2,asasd


In [64]:
df.replace(r'\s+',np.nan,regex=True)

,a
0,
1,NaN
2,asasd


In [104]:
x = ' '
x

' '

In [124]:
x.strip(' ')

''

In [108]:
[ord(c) for c in x]

[32]